In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

%matplotlib notebook

In [2]:
class Model(object):
    def __init__(self, train_acc_vs_t_function, init_train_duration):
        self.train_fn = train_acc_vs_t_function
        self.init_trained_duration = init_train_duration
        self.init_acc = train_acc_vs_t_function(init_train_duration)
        self.trained_duration = self.init_trained_duration
        self.acc = self.init_acc

    def post_train_acc(self, train_time):
        # self.trained_duration += train_time # Not updating this yet
        return self.train_fn(train_time + self.init_trained_duration)
    
    def do_train(self, train_time):
        self.trained_duration += train_time
        self.acc = self.train_fn(train_time)

def slowed_acc(acc, contention_slowdown = 0.9):
    return acc*contention_slowdown

def optimus_fn(t, T, scale):
    if t==0:
        return 0
    else:
        return scale*1/((1/(t*10*1/T)) + 1) * 100   # Hits 0.9 acc at time T. *100 for accuracy.

def inv_optimus_fn(a, T, scale):   # Maps accuracy to time
    if a==0:
        return 0
    else:
        return scale*a/(10/T * (100-a*scale))

def linear_fn(t, k):
    return k*t

def inv_linear_fn(a, k):
    return a/k

def get_linear_fn(k):
    return lambda t: linear_fn(t, k), lambda a: inv_linear_fn(a, k)

def get_optimus_fn(T, scale):
    return lambda t: optimus_fn(t, T, scale), lambda a: inv_optimus_fn(a, T, scale)

def get_AUC(train_models, first_duration, second_duration, T):
    if first_duration + second_duration >= T:
        return 0
    # Train_models is a list of models in the order they are trained
    assert len(train_models) == 2
    first_model = train_models[0]
    second_model = train_models[1]
    AUC_first = slowed_acc(first_model.init_acc) * first_duration + slowed_acc(first_model.post_train_acc(first_duration)) * second_duration + first_model.post_train_acc(first_duration) * (T-(first_duration + second_duration))
    AUC_second = slowed_acc(second_model.init_acc) * (first_duration + second_duration) + second_model.post_train_acc(second_duration) * (T-(first_duration + second_duration))
    return AUC_first + AUC_second

def get_AUC_curves(train_models, first_duration, second_duration, T):
    assert first_duration + second_duration <= T
    # Train_models is a list of models in the order they are trained
    assert len(train_models) == 2
    first_model = train_models[0]
    second_model = train_models[1]
    x_data = list(range(0, T))
    acc1 = []   # First accuracy curve
    acc2 = []
    for t in x_data:
        if t < first_duration:
            # Model 1 is training
            acc1.append(slowed_acc(first_model.init_acc))
            acc2.append(slowed_acc(second_model.init_acc))
        elif t >= first_duration and t < first_duration+second_duration:
            # Model 2 is training
            acc1.append(slowed_acc(first_model.post_train_acc(first_duration)))
            acc2.append(slowed_acc(second_model.init_acc))
        elif t >= first_duration + second_duration:
            # Both training done
            acc1.append(first_model.post_train_acc(first_duration))
            acc2.append(second_model.post_train_acc(second_duration))
    assert len(acc1) == len(acc2) == len(x_data)
    return acc1, acc2, x_data

def get_train_curves(train_models, first_duration, second_duration, T):
    assert first_duration + second_duration <= T
    # Train_models is a list of models in the order they are trained
    assert len(train_models) == 2
    first_model = train_models[0]
    second_model = train_models[1]
    x_data = list(np.arange(0.0, T, 0.1))
    acc1 = []   # First accuracy curve
    acc2 = []
    for t in x_data:
        if t < first_duration:
            # Model 1 is training
            acc1.append(first_model.post_train_acc(t))
            acc2.append(second_model.init_acc)
        elif t >= first_duration and t < first_duration+second_duration:
            # Model 2 is training
            acc1.append(first_model.post_train_acc(first_duration))
            acc2.append(second_model.post_train_acc(t-first_duration))
        elif t >= first_duration + second_duration:
            # Both training done
            acc1.append(first_model.post_train_acc(first_duration))
            acc2.append(second_model.post_train_acc(second_duration))
    assert len(acc1) == len(acc2) == len(x_data)
    return acc1, acc2, x_data

def plot_train_profiles(train_funcs, time_max=100, title="Accuracy vs GPU Cycles"):
    t_data_train = list(range(0,time_max))
    for fn, label in train_funcs:
        tacc = [fn(i) for i in t_data_train]
        plt.plot(t_data_train, tacc, label=label)
    plt.legend()
    plt.title(title)
    plt.xlabel("GPU Time")
    plt.ylabel("Accuracy")
    plt.ylim([0,100])

def plot_acc_data(lines, t_data, title="Accuracy over time"):
    for l in lines:
        acc_data, label = l
        plt.plot(t_data, acc_data, label=label)
    plt.legend()
    plt.title(title)
    plt.xlabel("Time")
    plt.ylabel("Accuracy")
    plt.ylim([0,100])

def plot_auc_plane(train_models, T):
    A,B = train_models
    train_times = list(range(0, T))
    first_train_time = []
    second_train_time = []
    auc_data = []
    for x in train_times:
        for y in range(0, T - x):
            first_train_time.append(x)
            second_train_time.append(y)
            auc_data.append(get_AUC([A, B], x, y, T))

    max_index = auc_data.index(max(auc_data))
    t1_opt, t2_opt, auc_opt = first_train_time[max_index], second_train_time[max_index], auc_data[max_index]
    print("Max coords: A: {}, B: {}, AUC: {}".format(t1_opt, t2_opt, auc_opt))

    fig = plt.figure()
    ax = fig.gca(projection='3d')
    # Plot the surface.
    surf = ax.scatter(first_train_time, second_train_time, auc_data)  # , linewidth=0, antialiased=False)
    ax.set_xlabel("A train time")
    ax.set_ylabel("B train time")
    ax.set_zlabel("AUC")
    ax.set_title("AUC vs A train time vs B train time")

    plt.figure()
    acc1, acc2, t_data = get_AUC_curves([A, B], t1_opt, t2_opt, T)
    plot_acc_data([(acc1, "FirstModel"), (acc2, "SecondModel")], t_data, title="Best Inference Accuracy over Time")
    print(sum(acc1) + sum(acc2))
    
    plt.figure()
    tacc1, tacc2, t_data_train = get_train_curves([A, B], t1_opt, t2_opt, T)
    plot_acc_data([(tacc1, "FirstModel_Train"), (tacc2, "SecondModel_Train")], t_data_train, title="Best Train Accuracy over Time")
    
    plt.figure()
    plot_train_profiles([(A.train_fn, "A_Train_Profile"), (B.train_fn, "B_Train_Profile")])
                        
    plt.show()

### Simulator start

In [3]:
def run_sim(a_conv_time = 50, a_scale = 1, b_conv_time = 10, b_scale = 0.8, target_start_accuracy = 30, T = 20):
    a_func, a_inv_func = get_optimus_fn(a_conv_time, a_scale)
    b_func, b_inv_func = get_optimus_fn(b_conv_time, b_scale)
    init_time_a = a_inv_func(target_start_accuracy)
    init_time_b = b_inv_func(target_start_accuracy)
    A = Model(a_func, init_time_a)
    B = Model(b_func, init_time_b)

    plot_auc_plane([A,B], T)
    #plot_auc_plane([B,A], T)

In [4]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [7]:
interactive_plot = interactive(run_sim, {'manual': True}, a_conv_time = (0, 500, 1), a_scale = (0,1,0.1), b_conv_time = (0, 500, 1), b_scale = (0,1,0.1), target_start_accuracy = 40, T = 20)
output = interactive_plot.children[-1]
#output.layout.height = '1200px'
interactive_plot


interactive(children=(IntSlider(value=50, description='a_conv_time', max=500), FloatSlider(value=1.0, descript…